# Top of notebook

In [2]:
import numpy as np
import glob
import time
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation


# %matplotlib widget
# %matplotlib inline
# %matplotlib widget
%matplotlib notebook

## Loading data and visualizations

The cell below loads a few npy files. They each have pose data from the original motinocapture files, however, **they are already
egocentered** such that the rat always looks to the right.

In [3]:
dnames = glob.glob('mocap_clean_eh5/*.npy')
egoh5s = []
for dname in dnames:
    egoh5s.append(np.load(dname, allow_pickle=True))

# egoh5s = [eh5.reshape((eh5.shape[0], eh5.shape[1]//3, 3)) for eh5 in egoh5s]

In [4]:
connections2 = [[5, 4, 6], [4,17,18], [11,12], [0,2,15,17,16,3,1], [13,9,7,18,8,10,14]]

fig, ax = plt.subplots()

def animfunc(t, skip=1):
    ax.clear()
    lines = []
    for conn in connections2:
        conn = [3*i for i in conn]
        lines.append(ax.plot(egoh5s[0][t*skip, conn], egoh5s[0][t*skip, [i+1 for i in conn]])[0])
    ax.set_title('(%i, %i)'%((t, t*skip)))
    return lines,

anim = FuncAnimation(fig, animfunc, frames=20, fargs=(1500,), repeat=False)
        
#anim.event_source.stop()        

<IPython.core.display.Javascript object>

Let's plot the center of mass of these poses. If they are not very erronous, they should all lie in a clumb near origin. 

In [5]:
np.argwhere(coms[:,0])

NameError: name 'coms' is not defined

In [6]:
ax.get_xlim()

(0.0, 1.0)

In [38]:
alleh5s = np.concatenate(egoh5s, axis=0)
alleh5s = alleh5s.reshape((-1, alleh5s.shape[1]//3, 3))
coms = np.mean(alleh5s, axis=1)


fig, ax = plt.subplots()
ax.scatter(coms[:,0], coms[:,1], s=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Zoom in in the figure above to enclose only the central blob. Then run the cell below to select points that are in the central blob.

In [48]:
xlim, ylim = ax.get_xlim(), ax.get_ylim()

inds = np.where((coms[:,0]<xlim[1]) & (coms[:,0]>xlim[0]) & (coms[:,1]<ylim[1]) & (coms[:,1]>ylim[0]))[0]


connections2 = [[5, 4, 6], [4,17,18], [11,12], [0,2,15,17,16,3,1], [13,9,7,18,8,10,14]]

fig2, ax2 = plt.subplots()

def animfunc(t):
    ax2.clear()
    lines = []
    for conn in connections2:        
        lines.append(ax2.plot(alleh5s[inds[t], conn, 0], alleh5s[inds[t], conn, 1])[0])
    ax2.set_title('(%i/%i, %i)'%((t, len(inds), inds[t])))
    ax2.set_xlim([-250, 250])
    ax2.set_ylim([-300, 300])
    return lines,

anim = FuncAnimation(fig2, animfunc, frames=len(inds), repeat=False)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [49]:
anim.event_source.stop()

So not all poses have a center of mass at origin, so we don't have a very clean pose data to start with. 

In [50]:
alleh5s_clean = alleh5s[inds]

## Training AE model 

Before moving on to a variationa autoencoder, let's try to train a simple autoencode, and see if it is able to reconstruct poses with sufficiently low error. 
This also gives opportunity to change different parameters (activation functions, number of units) for the overall network architecture.

In [100]:
alleh5s_clean.shape[0]/alleh5s.shape[0]

0.9994232202082776

In [189]:
trainx = alleh5s_clean[::30].reshape((-1, 57)).astype('float32')
valx = alleh5s_clean[15::60].reshape((-1, 57)).astype('float32')
print(trainx.shape, valx.shape)
print(trainx.min(), trainx.max())

(109858, 57) (54929, 57)
-224.95145 258.52106


In [53]:
import tensorflow as tf
import tensorflow.keras as k
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [229]:
AE = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape=(57,)),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(8, activation=None),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(57, activation=None)])

AE.compile('adam', 'mse')

In [230]:
AE.fit(trainx, trainx, batch_size=64, epochs=50, validation_data=(valx, valx), callbacks = [tf.keras.callbacks.ModelCheckpoint(
    'MotionCapture_AE.h5', monitor="val_loss", verbose=1, save_best_only=True)])

Epoch 1/50
1717/1717 [==============================] - 4s 2ms/step - loss: 304.7823 - val_loss: 35.5200

Epoch 00001: val_loss improved from inf to 35.51999, saving model to MotionCapture_AE.h5
Epoch 2/50
1717/1717 [==============================] - 4s 2ms/step - loss: 34.4955 - val_loss: 31.0455

Epoch 00002: val_loss improved from 35.51999 to 31.04551, saving model to MotionCapture_AE.h5
Epoch 3/50
1717/1717 [==============================] - 4s 2ms/step - loss: 30.0738 - val_loss: 26.7740

Epoch 00003: val_loss improved from 31.04551 to 26.77399, saving model to MotionCapture_AE.h5
Epoch 4/50
1717/1717 [==============================] - 4s 2ms/step - loss: 26.3440 - val_loss: 24.8504

Epoch 00004: val_loss improved from 26.77399 to 24.85043, saving model to MotionCapture_AE.h5
Epoch 5/50
1717/1717 [==============================] - 4s 2ms/step - loss: 24.7223 - val_loss: 22.2776

Epoch 00005: val_loss improved from 24.85043 to 22.27761, saving model to MotionCapture_AE.h5
Epoch 6/5

#all data AE error 70

#clean data AE error 22



            tf.keras.layers.InputLayer(input_shape=(57,)),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(8, activation='sigmoid'), #24 with 'tanh' here #16 with None!!!
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(57, activation=None)])



#clean data AE error 61



            tf.keras.layers.InputLayer(input_shape=(57,)),
            tf.keras.layers.Dense(32, activation=None),
            tf.keras.layers.Dense(16, activation=None),
            tf.keras.layers.Dense(8, activation='tanh'),
            tf.keras.layers.Dense(16, activation=None),
            tf.keras.layers.Dense(32, activation=None),
            tf.keras.layers.Dense(57, activation=None)])



#clean data AE error 41 with wonky activations - 



            tf.keras.layers.InputLayer(input_shape=(57,)),
            tf.keras.layers.Dense(32, activation='sigmoid'),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(8, activation='tanh'),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(32, activation='sigmoid'),
            tf.keras.layers.Dense(57, activation=None)])



#clean data AE error 38 with wonky activations - 



            tf.keras.layers.InputLayer(input_shape=(57,)),
            tf.keras.layers.Dense(32, activation='tanh'),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(8, activation='sigmoid'),
            tf.keras.layers.Dense(16, activation='tanh'),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(57, activation=None)])



#clean data AE error 66 with  - 



            tf.keras.layers.InputLayer(input_shape=(57,)),
            tf.keras.layers.Dense(32, activation='sigmoid'),
            tf.keras.layers.Dense(16, activation='sigmoid'),
            tf.keras.layers.Dense(8, activation='sigmoid'),
            tf.keras.layers.Dense(16, activation='sigmoid'),
            tf.keras.layers.Dense(32, activation='sigmoid'),
            tf.keras.layers.Dense(57, activation=None)])


In [231]:
valx_pred = AE.predict(valx, verbose=1)

1717/1717 [==============================] - 1s 681us/step


In [232]:
connections2 = [[5, 4, 6], [4,17,18], [11,12], [0,2,15,17,16,3,1], [13,9,7,18,8,10,14]]

fig, ax = plt.subplots()

def animfunc(t, skip=1):
    ax.clear()
    lines = []
    for conn in connections2:
        conn = [3*i for i in conn]
        lines.append(ax.plot(valx[t*skip, conn], valx[t*skip, [i+1 for i in conn]], color='royalblue')[0])
        lines.append(ax.plot(valx_pred[t*skip, conn], valx_pred[t*skip, [i+1 for i in conn]], color='firebrick')[0])
    ax.set_title('(%i, %i)'%((t, t*skip)))
    return lines,

anim = FuncAnimation(fig, animfunc, frames=50, fargs=(1000,), repeat=False)
        
#anim.event_source.stop()        

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Seems like the errors in the validation set are very close to less than 3px. That is great, 
especially since we did not normalize our data in any way. 

In [233]:
errors = np.abs(valx_pred-valx).reshape((-1, 19, 3))
np.mean(errors, axis=(0,1))

array([2.7490234, 2.8410492, 2.7641344], dtype=float32)

We can get the latent layer values by defining a new model with output from layer 2 of our autoencoder

In [368]:
AE.layers

In [236]:
AE2 = k.models.Model(inputs=AE.input, outputs=AE.layers[2].output)

In [237]:
#check if the weights are same 
[np.all(w1==w2) for w1,w2 in zip(AE.get_weights(),AE2.get_weights())]

[True, True, True, True, True, True]

In [238]:
alleh5s_clean_ae = AE2.predict(alleh5s_clean.reshape((-1, 57)).astype('float32'), verbose=1)
alleh5s_clean_ae.shape

102992/102992 [==============================] - 60s 585us/step


## Training a VAE Model

Now that we have some idea of what network architecture works for our data, we can try to add the variational inference 
inside the network. 

In [307]:
class Sampling(k.layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [365]:
class VAEModel(k.Model):
    def __init__(self, latent_dim=8, **kwargs):
        super(VAEModel, self).__init__(**kwargs)
        self.latent_dim = latent_dim
        self.encoder = self.create_encoder()
        self.decoder = self.create_decoder()
        self.total_loss_tracker = k.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = k.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = k.metrics.Mean(name="kl_loss")

    def create_encoder(self):
        encoder_input = tf.keras.Sequential([
                    tf.keras.layers.InputLayer(input_shape=(57,)),
                    tf.keras.layers.Dense(32, activation='relu'),
                   tf.keras.layers.Dense(16, activation='relu'),])

        z_mean = k.layers.Dense(self.latent_dim, name="z_mean", activation='tanh')(encoder_input.output)
        z_log_var = k.layers.Dense(self.latent_dim, name="z_log_var", activation='tanh')(encoder_input.output)

        z = Sampling()([z_mean, z_log_var])
        encoder = k.Model(encoder_input.input, [z_mean, z_log_var, z], name="encoder")        
        return encoder
    
    def create_decoder(self):
        decoder = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape=(self.latent_dim,)),
           tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(57)])
        return decoder

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                    k.losses.mean_squared_error(data, reconstruction))
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(kl_loss)
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def call(self, data):
        z_mean, z_log_var, z = self.encoder(data)
        reconstruction = self.decoder(z)
        return reconstruction

In [366]:
figloss, axloss = plt.subplots(figsize=(16, 8))
axloss2 = axloss.twinx()

class LossPlot(k.callbacks.Callback):
    def __init__(self):
        self.losses = [0,0,0]
        
    def on_epoch_end(self, epoch, logs=None):
        keys = list(logs.keys())
        self.losses = np.vstack([self.losses, [logs['loss'], logs['reconstruction_loss'], logs['kl_loss']]])
        axloss.clear()
        axloss.plot(self.losses[1:,0], '-', color='black', label='loss')
        axloss.plot(self.losses[1:,1], '-', color='royalblue', label='loss (recon)')
        axloss2.plot(self.losses[1:,2], '-', color='firebrick', label='loss (kl)')
        figloss.canvas.draw()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [367]:
vae = VAEModel(latent_dim=8)
vae.compile(optimizer=k.optimizers.Adam(learning_rate=0.001))
vae.fit(trainx, epochs=50, batch_size=64, workers=1, callbacks=LossPlot())

Epoch 1/50
1717/1717 [==============================] - 4s 2ms/step - loss: 743.9176 - reconstruction_loss: 442.6275 - kl_loss: 0.7584
Epoch 2/50
1717/1717 [==============================] - 4s 2ms/step - loss: 237.5290 - reconstruction_loss: 225.5734 - kl_loss: 0.7056
Epoch 3/50
1717/1717 [==============================] - 4s 2ms/step - loss: 202.7113 - reconstruction_loss: 194.2894 - kl_loss: 0.7028
Epoch 4/50
1717/1717 [==============================] - 4s 2ms/step - loss: 183.5383 - reconstruction_loss: 182.1982 - kl_loss: 0.7040
Epoch 5/50
1717/1717 [==============================] - 4s 2ms/step - loss: 179.6710 - reconstruction_loss: 175.8767 - kl_loss: 0.6940
Epoch 6/50
1717/1717 [==============================] - 4s 2ms/step - loss: 167.4005 - reconstruction_loss: 163.6813 - kl_loss: 0.6867
Epoch 7/50
1717/1717 [==============================] - 4s 2ms/step - loss: 155.4556 - reconstruction_loss: 151.8230 - kl_loss: 0.6850
Epoch 8/50
1717/1717 [==============================] -

In [369]:
valx_pred_vae = vae.predict(valx, verbose=1)

1717/1717 [==============================] - 1s 785us/step


In [370]:
connections2 = [[5, 4, 6], [4,17,18], [11,12], [0,2,15,17,16,3,1], [13,9,7,18,8,10,14]]

fig, ax = plt.subplots()

def animfunc(t, skip=1):
    ax.clear()
    lines = []
    for conn in connections2:
        conn = [3*i for i in conn]
        lines.append(ax.plot(valx[t*skip, conn], valx[t*skip, [i+1 for i in conn]], color='royalblue')[0])
        lines.append(ax.plot(valx_pred_vae[t*skip, conn], valx_pred_vae[t*skip, [i+1 for i in conn]], color='firebrick')[0])
    ax.set_title('(%i, %i)'%((t, t*skip)))
    return lines,

anim = FuncAnimation(fig, animfunc, frames=50, fargs=(1000,), repeat=False)
        
#anim.event_source.stop()        

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Seems like the errors in the validation set are around 7px (they were 3px w/ AE)

In [371]:
errors = np.abs(valx_pred_vae-valx).reshape((-1, 19, 3))
np.mean(errors, axis=(0,1))

array([7.718824, 7.4027  , 7.435683], dtype=float32)

The latent layer values can be obtained using the VAE 'encoder' sub-model. 

In [374]:
z_mean, z_log_var, z = vae.encoder.predict(alleh5s_clean.reshape((-1, 57)), verbose=1)

In [375]:
print(z_mean.shape, z_log_var.shape, z.shape)

(3295717, 8) (3295717, 8) (3295717, 8)
